In [1]:
import tensorflow as tf

# Clear any existing GPU settings
tf.keras.backend.clear_session()

# Set TensorFlow to run on GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Allow memory growth to prevent OOM (Out-of-Memory) errors
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is set up successfully!")
    except RuntimeError as e:
        print(f"GPU Setup Error: {e}")

2025-04-19 09:18:27.021328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745054307.216625      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745054307.269802      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU is set up successfully!


In [2]:
import tensorflow as tf
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from PIL import Image
import shutil

In [3]:
# Define dataset paths
dataset1_path = "/kaggle/input/lung-and-colon-cancer-histopathological-images/lung_colon_image_set/lung_image_sets"
dataset2_path = "/kaggle/input/iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset"
combined_dataset_path = "/kaggle/working/Combined_Lung_Dataset"

# Mapping folders to 3-class structure
datasets = {
    "lung_n": "Normal",
    "Normal cases": "Normal",
    "Bengin cases": "Benign",
    "lung_aca": "Malignant",
    "lung_scc": "Malignant",
    "Malignant cases": "Malignant",
}

# Make sure combined class folders exist
for category in set(datasets.values()):
    os.makedirs(os.path.join(combined_dataset_path, category), exist_ok=True)

# Function to copy & rename images with prefix
def copy_images(source_folder, target_folder, prefix):
    if os.path.exists(source_folder):
        for file in os.listdir(source_folder):
            if file.lower().endswith((".jpg", ".png", ".jpeg")):
                src_path = os.path.join(source_folder, file)
                new_filename = f"{prefix}_{file}"
                dst_path = os.path.join(target_folder, new_filename)
                shutil.copy(src_path, dst_path)

# Copy dataset 1 (H&E histology) → add "histo_" prefix
for folder, category in datasets.items():
    source_folder = os.path.join(dataset1_path, folder)
    target_folder = os.path.join(combined_dataset_path, category)
    copy_images(source_folder, target_folder, prefix="histo")

# Copy dataset 2 (CT scans) → add "ct_" prefix
for folder, category in datasets.items():
    source_folder = os.path.join(dataset2_path, folder)
    target_folder = os.path.join(combined_dataset_path, category)
    copy_images(source_folder, target_folder, prefix="ct")

# Cleanup: remove any empty folders
for category in set(datasets.values()):
    target_folder = os.path.join(combined_dataset_path, category)
    if len(os.listdir(target_folder)) == 0:
        print(f"Removing empty folder: {target_folder}")
        shutil.rmtree(target_folder)

print("✅ Dataset successfully merged into 3 classes: Normal, Benign, Malignant with histo/ct prefixes!")


✅ Dataset successfully merged into 3 classes: Normal, Benign, Malignant with histo/ct prefixes!


In [4]:
import os
import shutil

# === Paths ===
real_benign = "/kaggle/working/Combined_Lung_Dataset/Benign"
synthetic_benign = "/kaggle/input/gan-generated-ct-images"
target_non_malignant = "/kaggle/working/Combined_Lung_Dataset/Non-Malignant"

# Create target folder
os.makedirs(target_non_malignant, exist_ok=True)

# === Copy Real Benign Images (prefix: real_) ===
for filename in os.listdir(real_benign):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        src = os.path.join(real_benign, filename)
        dst = os.path.join(target_non_malignant, f"real_{filename}")
        shutil.copyfile(src, dst)

# === Copy Synthetic Benign Images (prefix: ct_synthetic_) ===
for filename in os.listdir(synthetic_benign):
    if filename.lower().endswith('.png'):  # All synthetic are .png
        src = os.path.join(synthetic_benign, filename)
        dst = os.path.join(target_non_malignant, f"ct_synthetic_{filename}")
        shutil.copyfile(src, dst)

print(f"✅ Combined Real + Synthetic Benign images into: {target_non_malignant}")
print(f"🧾 Total: {len(os.listdir(target_non_malignant))} images")


✅ Combined Real + Synthetic Benign images into: /kaggle/working/Combined_Lung_Dataset/Non-Malignant
🧾 Total: 5120 images


In [5]:
# === Define paths ===
root_dataset = "/kaggle/working/Combined_Lung_Dataset"
normal_dir = os.path.join(root_dataset, "Normal")
benign_combined_dir = os.path.join(root_dataset, "Non-Malignant")  # this contains real + synthetic Benign
malignant_dir = os.path.join(root_dataset, "Malignant")

# Final binary class folder
final_dataset = "/kaggle/working/Final_Binary_Dataset"
final_non_malignant = os.path.join(final_dataset, "Non-Malignant")
final_malignant = os.path.join(final_dataset, "Malignant")

# === Create directories ===
os.makedirs(final_non_malignant, exist_ok=True)
os.makedirs(final_malignant, exist_ok=True)

# === Copy Normal images to Non-Malignant ===
for file in os.listdir(normal_dir):
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        src = os.path.join(normal_dir, file)
        dst = os.path.join(final_non_malignant, f"normal_{file}")
        shutil.copyfile(src, dst)

# === Copy Benign (real + synthetic) to Non-Malignant ===
for file in os.listdir(benign_combined_dir):
    if file.lower().endswith('.png') or file.lower().endswith(('.jpg', '.jpeg')):
        src = os.path.join(benign_combined_dir, file)
        dst = os.path.join(final_non_malignant, file)
        shutil.copyfile(src, dst)

# === Copy Malignant images ===
for file in os.listdir(malignant_dir):
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        src = os.path.join(malignant_dir, file)
        dst = os.path.join(final_malignant, file)
        shutil.copyfile(src, dst)

print("✅ Final Binary Dataset structure created at /kaggle/working/Final_Binary_Dataset")
print(f"📁 Non-Malignant: {len(os.listdir(final_non_malignant))} images")
print(f"📁 Malignant: {len(os.listdir(final_malignant))} images")


✅ Final Binary Dataset structure created at /kaggle/working/Final_Binary_Dataset
📁 Non-Malignant: 10536 images
📁 Malignant: 10561 images


In [6]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

data_dir = "/kaggle/working/Final_Binary_Dataset"
np.random.seed(42)

image_paths = []
labels = []
synthetic_paths = []
synthetic_labels = []
label_dict = {}

# === Load image paths and separate synthetic ===
for idx, class_name in enumerate(sorted(os.listdir(data_dir))):
    class_dir = os.path.join(data_dir, class_name)
    if os.path.isdir(class_dir):
        label_dict[class_name] = idx
        for file in os.listdir(class_dir):
            full_path = os.path.join(class_dir, file)
            if class_name == "Non-Malignant" and file.startswith("synthetic_"):
                synthetic_paths.append(full_path)
                synthetic_labels.append(idx)
            else:
                image_paths.append(full_path)
                labels.append(idx)

image_paths = np.array(image_paths)
labels = np.array(labels)
synthetic_paths = np.array(synthetic_paths)
synthetic_labels = np.array(synthetic_labels)

# === Split real data into 70% train, 20% val, 10% test ===
X_temp, X_test, y_temp, y_test = train_test_split(
    image_paths, labels, test_size=0.1, stratify=labels, random_state=42)

X_train_real, X_val, y_train_real, y_val = train_test_split(
    X_temp, y_temp, test_size=2/9, stratify=y_temp, random_state=42)  # 2/9 of 90% → ~20%

# === Add synthetic data to training set only ===
X_train = np.concatenate((X_train_real, synthetic_paths))
y_train = np.concatenate((y_train_real, synthetic_labels))

# === Done ===
print("✅ 70/20/10 Split complete (synthetic used only in training).")
print(f"📦 Training Set: {len(X_train)} images (with synthetic)")
print(f"🧪 Validation Set: {len(X_val)} images (real only)")
print(f"🧫 Test Set: {len(X_test)} images (real only)")


✅ 70/20/10 Split complete (synthetic used only in training).
📦 Training Set: 14767 images (with synthetic)
🧪 Validation Set: 4220 images (real only)
🧫 Test Set: 2110 images (real only)


In [7]:
# Parameters
image_size = (256, 256)
mean = tf.convert_to_tensor([0.485, 0.456, 0.406], dtype=tf.float32)
std = tf.convert_to_tensor([0.229, 0.224, 0.225], dtype=tf.float32)

I0000 00:00:1745054486.576450      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [8]:
@tf.function
def augment_image(image, label, filename):
    # Apply base spatial transforms to all images
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    if tf.random.uniform([]) > 0.7:
        k = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
        image = tf.image.rot90(image, k)

    # === Conditional Augmentations ===
    # For histopathology images (H&E)
    if tf.strings.regex_full_match(filename, ".*histo.*|.*real.*"):
        if tf.random.uniform([]) > 0.7:
            image = tf.image.random_brightness(image, max_delta=0.08)
        if tf.random.uniform([]) > 0.7:
            image = tf.image.random_contrast(image, 0.9, 1.1)
        if tf.random.uniform([]) > 0.7:
            image = tf.image.random_saturation(image, 0.9, 1.1)
        if tf.random.uniform([]) > 0.9:
            image = tf.image.random_jpeg_quality(image, 80, 100)
        if tf.random.uniform([]) > 0.7:
            image = tf.image.central_crop(image, 0.9)
            image = tf.image.resize(image, [256, 256])
        if tf.random.uniform([]) > 0.7:
            noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.01)
            image = tf.clip_by_value(image + noise, 0.0, 1.0)

    # For CT/synthetic images
    elif tf.strings.regex_full_match(filename, ".*ct.*|.*synthetic.*"):
        # Less aggressive augmentations for CT
        if tf.random.uniform([]) > 0.8:
            image = tf.image.central_crop(image, 0.95)
            image = tf.image.resize(image, [256, 256])
        if tf.random.uniform([]) > 0.8:
            # Slight noise for CT images
            noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.002)
            image = tf.clip_by_value(image + noise, 0.0, 1.0)
        if tf.random.uniform([]) > 0.2:  # Small probability for brightness change for CT
            image = tf.image.random_brightness(image, max_delta=0.02)  # Small brightness for CT scans

    return image, label


In [9]:
# ================= Preprocessing Function ================= #
def load_and_preprocess_image(image_path, label, augment=False):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, image_size)
    print("Before normalization:", tf.reduce_min(img), tf.reduce_max(img))
    img = img / 255.0  # Normalize pixel values
    print("After division by 255:", tf.reduce_min(img), tf.reduce_max(img))
    # Apply augmentation only if augment=True
    if augment:
        img, label = augment_image(img, label, filename)
    print("After mean/std normalization:", tf.reduce_min(img), tf.reduce_max(img))
    img = (img - mean) / std  # Apply mean-std normalization
    img = tf.cast(img, tf.float32)
    return img, label

In [10]:
# ================= Dataset Creation ================= #
def create_dataset(image_paths, labels, batch_size=32, shuffle=True, augment=False):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda x, y: load_and_preprocess_image(x, y, augment), num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(len(image_paths))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [11]:
train_dataset = create_dataset(X_train, y_train, augment=True)  # Augmentation only in training
val_dataset = create_dataset(X_val, y_val, shuffle=False)
test_dataset = create_dataset(X_test, y_test, shuffle=False)

Before normalization: Tensor("Min:0", shape=(), dtype=float32) Tensor("Max:0", shape=(), dtype=float32)
After division by 255: Tensor("Min_1:0", shape=(), dtype=float32) Tensor("Max_1:0", shape=(), dtype=float32)
After mean/std normalization: Tensor("Min_2:0", shape=(), dtype=float32) Tensor("Max_2:0", shape=(), dtype=float32)
Before normalization: Tensor("Min:0", shape=(), dtype=float32) Tensor("Max:0", shape=(), dtype=float32)
After division by 255: Tensor("Min_1:0", shape=(), dtype=float32) Tensor("Max_1:0", shape=(), dtype=float32)
After mean/std normalization: Tensor("Min_2:0", shape=(), dtype=float32) Tensor("Max_2:0", shape=(), dtype=float32)
Before normalization: Tensor("Min:0", shape=(), dtype=float32) Tensor("Max:0", shape=(), dtype=float32)
After division by 255: Tensor("Min_1:0", shape=(), dtype=float32) Tensor("Max_1:0", shape=(), dtype=float32)
After mean/std normalization: Tensor("Min_2:0", shape=(), dtype=float32) Tensor("Max_2:0", shape=(), dtype=float32)


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, mixed_precision
import numpy as np
from tensorflow.keras.models import load_model

In [13]:
# Step 2: Enable mixed precision (optional, if the model was trained with mixed precision)
policy = mixed_precision.Policy('mixed_float16')  # Set mixed precision policy
mixed_precision.set_global_policy(policy)

In [14]:
# Step 1: Define the Cast custom layer
class Cast(layers.Layer):
    def __init__(self, dtype='float32', **kwargs):
        super(Cast, self).__init__(**kwargs)
        self._dtype = dtype  # Store dtype as a regular attribute (not a property)

    def call(self, inputs):
        # Cast the inputs to the desired dtype (use self._dtype here)
        return tf.cast(inputs, self._dtype)

    def get_config(self):
        # Include dtype in the layer's configuration for proper deserialization
        config = super(Cast, self).get_config()
        config.update({'dtype': self._dtype})  # Add dtype to the config dictionary
        return config

In [15]:
'''
# Remove the last two layers (Global Average Pooling and Dense layer)
resnet150_model = tf.keras.Model(
    inputs=resnet150_model.input, 
    outputs=resnet150_model.layers[-3].output  # Exclude the last 2 layers (GAP and Dense)
)

# Create a dummy input tensor (e.g., 256x256 RGB image)
dummy_input = tf.random.normal((1, 256, 256, 3))  # 1 image of size 256x256x3

# Pass the dummy input through the modified model
resnet_output = resnet150_model(dummy_input)
print(f"Feature Vector Shape: {resnet_output.shape}")
'''

'\n# Remove the last two layers (Global Average Pooling and Dense layer)\nresnet150_model = tf.keras.Model(\n    inputs=resnet150_model.input, \n    outputs=resnet150_model.layers[-3].output  # Exclude the last 2 layers (GAP and Dense)\n)\n\n# Create a dummy input tensor (e.g., 256x256 RGB image)\ndummy_input = tf.random.normal((1, 256, 256, 3))  # 1 image of size 256x256x3\n\n# Pass the dummy input through the modified model\nresnet_output = resnet150_model(dummy_input)\nprint(f"Feature Vector Shape: {resnet_output.shape}")\n'

In [16]:
densenet121_model = tf.keras.models.load_model(
    '/kaggle/input/densenet121_model/keras/default/1/densenet121_lung_model.h5',
    custom_objects={'Cast': Cast},  # Register the custom Cast layer
)


In [17]:
#Load the resnet150_model model with the custom Cast layer
resnet150_model = tf.keras.models.load_model(
    '/kaggle/input/resnet150_lung_model/keras/default/1/resnet150_model .h5',  # Update with your model path
    custom_objects={'Cast': Cast},  # Register the custom Cast layer
)


In [18]:
resnet50_model = tf.keras.models.load_model(
    '/kaggle/input/resnet50/keras/default/1/resnet50_lung_model.h5',  # Update with your model path
    custom_objects={'Cast': Cast},  # Register the custom Cast layer
)

In [19]:
efficientnetb0_model = tf.keras.models.load_model(
    '/kaggle/input/efficientnetb0_lung_model/keras/default/1/efficientnet_b0_lung_model.h5',  # Path to your model
    custom_objects={'Cast': Cast},  # Register the custom Cast layer
)

In [20]:
efficientnetb1_model = tf.keras.models.load_model(
    '/kaggle/input/efficientnetb1/keras/default/1/efficientnet_b1_lung_model.h5',  # Update with your model path
    custom_objects={'Cast': Cast},  # Register the custom Cast layer
)

In [21]:
import numpy as np
import tensorflow as tf

# Assuming test_dataset is already created and loaded earlier
# test_dataset is assumed to be a tf.data.Dataset or ImageDataGenerator (not reloaded here again)

# Predict using DenseNet121 (logits are returned, not probabilities)
densenet_predictions = densenet121_model.predict(test_dataset, verbose=1)

# Predict using ResNet150 (logits are returned, not probabilities)
resnet150_predictions = resnet150_model.predict(test_dataset, verbose=1)

# Print the first few predictions to check the logits
print("DenseNet121 Predictions (first 5 logits):")
print(densenet_predictions[:5])

print("ResNet150 Predictions (first 5 logits):")
print(resnet150_predictions[:5])

# Apply sigmoid to convert logits to probabilities
densenet_probabilities = tf.sigmoid(densenet_predictions).numpy()
resnet150_probabilities = tf.sigmoid(resnet150_predictions).numpy()

# Average the probabilities from both models (simple averaging)
ensemble_predictions = (densenet_probabilities + resnet150_probabilities) / 2

# Print the first few ensemble predictions (probabilities)
print("Ensemble Predictions (first 5 probabilities):")
print(ensemble_predictions[:5])

# Convert probabilities to binary predictions (thresholding at 0.5 for binary classification)
ensemble_binary_predictions = (ensemble_predictions > 0.5).astype(int)

# Extract true labels from test_dataset
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Print the first few true labels to check
print("True Labels (first 5):")
print(true_labels[:5])

# Evaluate the ensemble model accuracy
accuracy = np.mean(ensemble_binary_predictions == true_labels)
print(f'Ensemble Model Accuracy: {accuracy * 100:.2f}%')


I0000 00:00:1745054504.693293      86 service.cc:148] XLA service 0x7fbb9c002ef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745054504.694091      86 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1745054506.028475      86 cuda_dnn.cc:529] Loaded cuDNN version 90300


 3/66 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step  

I0000 00:00:1745054517.352745      86 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


66/66 ━━━━━━━━━━━━━━━━━━━━ 38s 307ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 10s 101ms/step
DenseNet121 Predictions (first 5 logits):
[[2.216e-04]
 [1.000e+00]
 [9.990e-01]
 [2.157e-03]
 [1.000e+00]]
ResNet150 Predictions (first 5 logits):
[[5.7230256e-05]
 [9.9997306e-01]
 [9.9970502e-01]
 [8.1271879e-05]
 [9.9994695e-01]]
Ensemble Predictions (first 5 probabilities):
[[0.50000715]
 [0.73100513]
 [0.7309788 ]
 [0.5002543 ]
 [0.73100257]]
True Labels (first 5):
[0 1 1 0 1]
Ensemble Model Accuracy: 49.95%


In [22]:
import numpy as np
import tensorflow as tf

# Assuming test_dataset is already created and loaded earlier
# test_dataset is assumed to be a tf.data.Dataset or ImageDataGenerator (not reloaded here again)

# Predict using DenseNet121 (logits are returned, not probabilities)
densenet_predictions = densenet121_model.predict(test_dataset, verbose=1)

# Predict using ResNet150 (logits are returned, not probabilities)
resnet150_predictions = resnet150_model.predict(test_dataset, verbose=1)

# Apply sigmoid to convert logits to probabilities
densenet_probabilities = tf.sigmoid(densenet_predictions).numpy()
resnet150_probabilities = tf.sigmoid(resnet150_predictions).numpy()

# Weighted averaging (assuming DenseNet121 performs better)
weight_densenet = 0.7  # Assigning higher weight to DenseNet121
weight_resnet = 0.3    # Assigning lower weight to ResNet150

# Average the predictions using weighted averages
ensemble_predictions = (weight_densenet * densenet_probabilities + weight_resnet * resnet150_probabilities)

# Print the first few ensemble predictions (probabilities)
print("Ensemble Predictions (first 5 probabilities):")
print(ensemble_predictions[:5])

# Convert probabilities to binary predictions (thresholding at 0.5 for binary classification)
ensemble_binary_predictions = (ensemble_predictions > 0.5).astype(int)

# Extract true labels from test_dataset
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Print the first few true labels to check
print("True Labels (first 5):")
print(true_labels[:5])

# Evaluate the ensemble model accuracy
accuracy = np.mean(ensemble_binary_predictions == true_labels)
print(f'Ensemble Model Accuracy: {accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step
Ensemble Predictions (first 5 probabilities):
[[0.5001019 ]
 [0.73103476]
 [0.7310189 ]
 [0.5003479 ]
 [0.7310332 ]]
True Labels (first 5):
[0 1 1 0 1]
Ensemble Model Accuracy: 49.95%


In [23]:
import numpy as np
import tensorflow as tf

# Assuming test_dataset is already created and loaded earlier

# Predict using DenseNet121 (logits are returned, not probabilities)
densenet_predictions = densenet121_model.predict(test_dataset, verbose=1)

# Predict using ResNet150 (logits are returned, not probabilities)
resnet150_predictions = resnet150_model.predict(test_dataset, verbose=1)

# Apply sigmoid to convert logits to probabilities
densenet_probabilities = tf.sigmoid(densenet_predictions).numpy()
resnet150_probabilities = tf.sigmoid(resnet150_predictions).numpy()

# Weighted averaging (assuming DenseNet121 performs better)
weight_densenet = 0.7  # Assigning higher weight to DenseNet121
weight_resnet = 0.3    # Assigning lower weight to ResNet150

# Average the predictions using weighted averages
ensemble_predictions = (weight_densenet * densenet_probabilities + weight_resnet * resnet150_probabilities)

# Print the first few ensemble predictions (probabilities)
print("Ensemble Predictions (first 5 probabilities):")
print(ensemble_predictions[:5])

# Increase threshold to 0.7 for better confidence
threshold = 0.7
ensemble_binary_predictions = (ensemble_predictions > threshold).astype(int)

# Extract true labels from test_dataset
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Print the first few true labels to check
print("True Labels (first 5):")
print(true_labels[:5])

# Evaluate the ensemble model accuracy
accuracy = np.mean(ensemble_binary_predictions == true_labels)
print(f'Ensemble Model Accuracy: {accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step
Ensemble Predictions (first 5 probabilities):
[[0.5001019 ]
 [0.73103476]
 [0.7310189 ]
 [0.5003479 ]
 [0.7310332 ]]
True Labels (first 5):
[0 1 1 0 1]
Ensemble Model Accuracy: 50.00%


In [24]:
import numpy as np
import tensorflow as tf

# Assuming test_dataset is already created and loaded earlier

# Predict using DenseNet121 (logits are returned, not probabilities)
densenet_predictions = densenet121_model.predict(test_dataset, verbose=1)

# Predict using ResNet150 (logits are returned, not probabilities)
resnet150_predictions = resnet150_model.predict(test_dataset, verbose=1)

# Apply sigmoid to convert logits to probabilities
densenet_probabilities = tf.sigmoid(densenet_predictions).numpy()
resnet150_probabilities = tf.sigmoid(resnet150_predictions).numpy()

# Weighted averaging (assuming DenseNet121 performs better)
weight_densenet = 0.7  # Assigning higher weight to DenseNet121
weight_resnet = 0.3    # Assigning lower weight to ResNet150

# Average the predictions using weighted averages
ensemble_predictions = (weight_densenet * densenet_probabilities + weight_resnet * resnet150_probabilities)

# Print the first few ensemble predictions (probabilities)
print("Ensemble Predictions (first 5 probabilities):")
print(ensemble_predictions[:5])

# Lower the threshold for more confident predictions
threshold = 0.6  # Try a lower threshold for binary classification
ensemble_binary_predictions = (ensemble_predictions > threshold).astype(int)

# Extract true labels from test_dataset
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Print the first few true labels to check
print("True Labels (first 5):")
print(true_labels[:5])

# Evaluate the ensemble model accuracy
accuracy = np.mean(ensemble_binary_predictions == true_labels)
print(f'Ensemble Model Accuracy: {accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step
Ensemble Predictions (first 5 probabilities):
[[0.5001019 ]
 [0.73103476]
 [0.7310189 ]
 [0.5003479 ]
 [0.7310332 ]]
True Labels (first 5):
[0 1 1 0 1]
Ensemble Model Accuracy: 50.00%


hard viting

In [25]:
import numpy as np
import tensorflow as tf

# Assuming test_dataset is already created and loaded earlier

# Predict using DenseNet121 (logits are returned, not probabilities)
densenet_predictions = densenet121_model.predict(test_dataset, verbose=1)

# Predict using ResNet150 (logits are returned, not probabilities)
resnet150_predictions = resnet150_model.predict(test_dataset, verbose=1)

# Apply sigmoid to convert logits to probabilities
densenet_probabilities = tf.sigmoid(densenet_predictions).numpy()
resnet150_probabilities = tf.sigmoid(resnet150_predictions).numpy()

# Convert probabilities to binary predictions (thresholding at 0.5 for binary classification)
densenet_binary_predictions = (densenet_probabilities > 0.5).astype(int)
resnet150_binary_predictions = (resnet150_probabilities > 0.5).astype(int)

# Majority voting for the ensemble (for each sample, choose the majority vote between the two models)
ensemble_binary_predictions = np.round((densenet_binary_predictions + resnet150_binary_predictions) / 2)

# Print the first few ensemble predictions
print("Ensemble Predictions (first 5):")
print(ensemble_binary_predictions[:5])

# Extract true labels from test_dataset
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Print the first few true labels to check
print("True Labels (first 5):")
print(true_labels[:5])

# Evaluate the ensemble model accuracy
accuracy = np.mean(ensemble_binary_predictions == true_labels)
print(f'Ensemble Model Accuracy: {accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step
Ensemble Predictions (first 5):
[[0.]
 [1.]
 [1.]
 [1.]
 [1.]]
True Labels (first 5):
[0 1 1 0 1]
Ensemble Model Accuracy: 49.99%


weight

In [26]:
import numpy as np
import tensorflow as tf

# Assuming test_dataset is already created and loaded earlier

# Predict using DenseNet121 (logits are returned, not probabilities)
densenet_predictions = densenet121_model.predict(test_dataset, verbose=1)

# Predict using ResNet150 (logits are returned, not probabilities)
resnet150_predictions = resnet150_model.predict(test_dataset, verbose=1)

# Apply sigmoid to convert logits to probabilities
densenet_probabilities = tf.sigmoid(densenet_predictions).numpy()
resnet150_probabilities = tf.sigmoid(resnet150_predictions).numpy()

# Convert probabilities to binary predictions (thresholding at 0.5 for binary classification)
densenet_binary_predictions = (densenet_probabilities > 0.5).astype(int)
resnet150_binary_predictions = (resnet150_probabilities > 0.5).astype(int)

# Weighted Majority Voting: Give more weight to DenseNet121 if it's performing better
weight_densenet = 0.7  # Assign more weight to DenseNet121
weight_resnet = 0.3    # Assign less weight to ResNet150

# Weighted sum of the binary predictions
ensemble_binary_predictions = np.round((weight_densenet * densenet_binary_predictions + weight_resnet * resnet150_binary_predictions))

# Alternatively, you can use soft voting (averaging probabilities)
# ensemble_predictions = (densenet_probabilities + resnet150_probabilities) / 2
# ensemble_binary_predictions = (ensemble_predictions > 0.5).astype(int)

# Print the first few ensemble predictions
print("Ensemble Predictions (first 5):")
print(ensemble_binary_predictions[:5])

# Extract true labels from test_dataset
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Print the first few true labels to check
print("True Labels (first 5):")
print(true_labels[:5])

# Evaluate the ensemble model accuracy
accuracy = np.mean(ensemble_binary_predictions == true_labels)
print(f'Ensemble Model Accuracy: {accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step
Ensemble Predictions (first 5):
[[0.]
 [1.]
 [1.]
 [1.]
 [1.]]
True Labels (first 5):
[0 1 1 0 1]
Ensemble Model Accuracy: 49.99%


soft voting:

In [27]:
import numpy as np
import tensorflow as tf

# Assuming test_dataset is already created and loaded earlier

# Predict using DenseNet121 (logits are returned, not probabilities)
densenet_predictions = densenet121_model.predict(test_dataset, verbose=1)

# Predict using ResNet150 (logits are returned, not probabilities)
resnet150_predictions = resnet150_model.predict(test_dataset, verbose=1)

# Apply sigmoid to convert logits to probabilities
densenet_probabilities = tf.sigmoid(densenet_predictions).numpy()
resnet150_probabilities = tf.sigmoid(resnet150_predictions).numpy()

# Soft Voting: Average the probabilities from both models
ensemble_predictions = (densenet_probabilities + resnet150_probabilities) / 2

# Apply thresholding at 0.5 for binary classification
ensemble_binary_predictions = (ensemble_predictions > 0.5).astype(int)

# Print the first few ensemble predictions
print("Ensemble Predictions (first 5):")
print(ensemble_binary_predictions[:5])

# Extract true labels from test_dataset
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Print the first few true labels to check
print("True Labels (first 5):")
print(true_labels[:5])

# Evaluate the ensemble model accuracy
accuracy = np.mean(ensemble_binary_predictions == true_labels)
print(f'Ensemble Model Accuracy: {accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step
Ensemble Predictions (first 5):
[[1]
 [1]
 [1]
 [1]
 [1]]
True Labels (first 5):
[0 1 1 0 1]
Ensemble Model Accuracy: 49.95%


threshold and weight

In [28]:
import numpy as np
import tensorflow as tf

# Assuming test_dataset is already created and loaded earlier
# test_dataset is assumed to be a tf.data.Dataset or ImageDataGenerator (not reloaded here again)

# Predict using DenseNet121 (logits are returned, not probabilities)
densenet_predictions = densenet121_model.predict(test_dataset, verbose=1)

# Predict using ResNet50 (logits are returned, not probabilities)
resnet50_predictions = resnet50_model.predict(test_dataset, verbose=1)

# Print the first few predictions to check the logits
print("DenseNet121 Predictions (first 5 logits):")
print(densenet_predictions[:5])

print("ResNet150 Predictions (first 5 logits):")
print(resnet50_predictions[:5])

# Apply sigmoid to convert logits to probabilities
densenet_probabilities = tf.sigmoid(densenet_predictions).numpy()
resnet50_probabilities = tf.sigmoid(resnet50_predictions).numpy()

# Average the probabilities from both models (simple averaging)
ensemble_predictions = (densenet_probabilities + resnet50_probabilities) / 2

# Print the first few ensemble predictions (probabilities)
print("Ensemble Predictions (first 5 probabilities):")
print(ensemble_predictions[:5])

# Convert probabilities to binary predictions (thresholding at 0.5 for binary classification)
ensemble_binary_predictions = (ensemble_predictions > 0.5).astype(int)

# Extract true labels from test_dataset
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Print the first few true labels to check
print("True Labels (first 5):")
print(true_labels[:5])

# Evaluate the ensemble model accuracy
accuracy = np.mean(ensemble_binary_predictions == true_labels)
print(f'Ensemble Model Accuracy: {accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 16s 146ms/step
DenseNet121 Predictions (first 5 logits):
[[2.216e-04]
 [1.000e+00]
 [9.990e-01]
 [2.157e-03]
 [1.000e+00]]
ResNet150 Predictions (first 5 logits):
[[0.001518]
 [1.      ]
 [0.998   ]
 [0.03119 ]
 [1.      ]]
Ensemble Predictions (first 5 probabilities):
[[0.5   ]
 [0.731 ]
 [0.7305]
 [0.504 ]
 [0.731 ]]
True Labels (first 5):
[0 1 1 0 1]
Ensemble Model Accuracy: 49.99%


In [29]:
import numpy as np
import tensorflow as tf

# Assuming test_dataset is already created and loaded earlier
# test_dataset is assumed to be a tf.data.Dataset or ImageDataGenerator (not reloaded here again)

# Predict using DenseNet121 (logits are returned, not probabilities)
densenet_predictions = densenet121_model.predict(test_dataset, verbose=1)

# Predict using ResNet150 (logits are returned, not probabilities)
resnet50_predictions = resnet50_model.predict(test_dataset, verbose=1)

# Apply sigmoid to convert logits to probabilities
densenet_probabilities = tf.sigmoid(densenet_predictions).numpy()
resnet50_probabilities = tf.sigmoid(resnet50_predictions).numpy()

# Weighted averaging (assuming DenseNet121 performs better)
weight_densenet = 0.7  # Assigning higher weight to DenseNet121
weight_resnet = 0.3    # Assigning lower weight to ResNet150

# Average the predictions using weighted averages
ensemble_predictions = (weight_densenet * densenet_probabilities + weight_resnet * resnet50_probabilities)

# Print the first few ensemble predictions (probabilities)
print("Ensemble Predictions (first 5 probabilities):")
print(ensemble_predictions[:5])

# Convert probabilities to binary predictions (thresholding at 0.5 for binary classification)
ensemble_binary_predictions = (ensemble_predictions > 0.5).astype(int)

# Extract true labels from test_dataset
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Print the first few true labels to check
print("True Labels (first 5):")
print(true_labels[:5])

# Evaluate the ensemble model accuracy
accuracy = np.mean(ensemble_binary_predictions == true_labels)
print(f'Ensemble Model Accuracy: {accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step
Ensemble Predictions (first 5 probabilities):
[[0.5  ]
 [0.731]
 [0.731]
 [0.503]
 [0.731]]
True Labels (first 5):
[0 1 1 0 1]
Ensemble Model Accuracy: 49.99%


In [30]:
import numpy as np
import tensorflow as tf

# Assuming models are already loaded (DenseNet121, ResNet150, EfficientNetB0)

# ==================== Predictions from Models ==================== #
# Get predictions for each model on the test dataset
densenet121_predictions = densenet121_model.predict(test_dataset, verbose=1)
resnet150_predictions = resnet150_model.predict(test_dataset, verbose=1)
efficientnet_predictions = efficientnetb0_model.predict(test_dataset, verbose=1)
# Print first few predictions from each model to check for diversity
print("DenseNet121 Predictions (first 5):")
print(densenet121_predictions[:5])

print("ResNet150 Predictions (first 5):")
print(resnet150_predictions[:5])

print("EfficientNet Predictions (first 5):")
print(efficientnet_predictions[:5])

# ==================== Convert Logits to Probabilities (if necessary) ==================== #
# If models output logits, apply sigmoid to get probabilities
densenet121_probabilities = tf.sigmoid(densenet121_predictions).numpy()
resnet150_probabilities = tf.sigmoid(resnet150_predictions).numpy()
efficientnet_probabilities = tf.sigmoid(efficientnet_predictions).numpy()

# Print first few probabilities to see the model output
print("DenseNet121 Probabilities (first 5):")
print(densenet121_probabilities[:5])

print("ResNet150 Probabilities (first 5):")
print(resnet150_probabilities[:5])

print("EfficientNet Probabilities (first 5):")
print(efficientnet_probabilities[:5])

# ==================== Weighted Averaging of Probabilities ==================== #
# Weights for each model (adjust based on performance)
# Since DenseNet121 and ResNet150 show similar performance, we give them equal weight
weight_densenet = 0.2  # Reduced weight for DenseNet121
weight_resnet = 0.2    # Reduced weight for ResNet150
weight_efficientnet = 0.6  # Increased weight for EfficientNet (if it's performing better)

# Average the probabilities with weights
ensemble_probabilities = (
    weight_densenet * densenet121_probabilities + 
    weight_resnet * resnet150_probabilities + 
    weight_efficientnet * efficientnet_probabilities
) / (weight_densenet + weight_resnet + weight_efficientnet)

# Print first few ensemble probabilities to check the output
print("Ensemble Probabilities (first 5):")
print(ensemble_probabilities[:5])

# ==================== Convert Probabilities to Binary Predictions ==================== #
# Use a threshold of 0.5 to decide final prediction
ensemble_binary_predictions = (ensemble_probabilities > 0.5).astype(int)

# Print first few ensemble binary predictions
print("Ensemble Binary Predictions (first 5):")
print(ensemble_binary_predictions[:5])

# ==================== Evaluate Ensemble Model ==================== #
# Extract true labels from test_dataset
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Calculate accuracy of the ensemble model
accuracy = np.mean(ensemble_binary_predictions == true_labels)

# Print the accuracy of the ensemble model
print(f'Ensemble Model Accuracy: {accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 19s 177ms/step
DenseNet121 Predictions (first 5):
[[2.216e-04]
 [1.000e+00]
 [9.990e-01]
 [2.157e-03]
 [1.000e+00]]
ResNet150 Predictions (first 5):
[[5.7230256e-05]
 [9.9997306e-01]
 [9.9970502e-01]
 [8.1271879e-05]
 [9.9994695e-01]]
EfficientNet Predictions (first 5):
[[0.00144779]
 [0.99391127]
 [0.9653318 ]
 [0.04704069]
 [0.6775769 ]]
DenseNet121 Probabilities (first 5):
[[0.5   ]
 [0.731 ]
 [0.731 ]
 [0.5005]
 [0.731 ]]
ResNet150 Probabilities (first 5):
[[0.5000143 ]
 [0.73105323]
 [0.73100054]
 [0.5000203 ]
 [0.73104817]]
EfficientNet Probabilities (first 5):
[[0.5003619 ]
 [0.72985977]
 [0.7241881 ]
 [0.51175797]
 [0.66319764]]
Ensemble Probabilities (first 5):
[[0.5001956 ]
 [0.73024476]
 [0.7268312 ]
 [0.5071565 ]
 [0.69024646]]
Ensemble Binary Predictions (first 5):
[[1]
 [1]
 [1]
 [1]
 [1]]
Ensemble Model Accuracy: 49.97%


In [31]:
import numpy as np
import tensorflow as tf

# Assuming models are already loaded (DenseNet121, ResNet150, EfficientNetB0)

# ==================== Predictions from Models ==================== #
# Get predictions for each model on the test dataset
densenet121_predictions = densenet121_model.predict(test_dataset, verbose=1)
resnet150_predictions = resnet150_model.predict(test_dataset, verbose=1)
efficientnet_predictions = efficientnetb0_model.predict(test_dataset, verbose=1)

# Print first few predictions from each model to check for diversity
print("DenseNet121 Predictions (first 5):")
print(densenet121_predictions[:5])

print("ResNet150 Predictions (first 5):")
print(resnet150_predictions[:5])

print("EfficientNet Predictions (first 5):")
print(efficientnet_predictions[:5])

# ==================== Convert Logits to Probabilities (if necessary) ==================== #
# If models output logits, apply sigmoid to get probabilities
densenet121_probabilities = tf.sigmoid(densenet121_predictions).numpy()
resnet150_probabilities = tf.sigmoid(resnet150_predictions).numpy()
efficientnet_probabilities = tf.sigmoid(efficientnet_predictions).numpy()

# Print first few probabilities to see the model output
print("DenseNet121 Probabilities (first 5):")
print(densenet121_probabilities[:5])

print("ResNet150 Probabilities (first 5):")
print(resnet150_probabilities[:5])

print("EfficientNet Probabilities (first 5):")
print(efficientnet_probabilities[:5])

# ==================== Weighted Averaging of Probabilities ==================== #
# Weights for each model (adjust based on performance)
# Since DenseNet121 and ResNet150 show similar performance, we give them equal weight
weight_densenet = 0.2  # Reduced weight for DenseNet121
weight_resnet = 0.2    # Reduced weight for ResNet150
weight_efficientnet = 0.6  # Increased weight for EfficientNet (if it's performing better)

# Average the probabilities with weights
ensemble_probabilities = (
    weight_densenet * densenet121_probabilities + 
    weight_resnet * resnet150_probabilities + 
    weight_efficientnet * efficientnet_probabilities
) / (weight_densenet + weight_resnet + weight_efficientnet)

# Print first few ensemble probabilities to check the output
print("Ensemble Probabilities (first 5):")
print(ensemble_probabilities[:5])

# ==================== Convert Probabilities to Binary Predictions ==================== #
# Use a threshold of 0.5 to decide final prediction
ensemble_binary_predictions = (ensemble_probabilities > 0.5).astype(int)

# Print first few ensemble binary predictions
print("Ensemble Binary Predictions (first 5):")
print(ensemble_binary_predictions[:5])

# ==================== Evaluate Ensemble Model ==================== #
# Extract true labels from test_dataset
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Calculate accuracy of the ensemble model
accuracy = np.mean(ensemble_binary_predictions == true_labels)

# Print the accuracy of the ensemble model
print(f'Ensemble Model Accuracy: {accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step
DenseNet121 Predictions (first 5):
[[2.216e-04]
 [1.000e+00]
 [9.990e-01]
 [2.157e-03]
 [1.000e+00]]
ResNet150 Predictions (first 5):
[[5.7230256e-05]
 [9.9997306e-01]
 [9.9970502e-01]
 [8.1271879e-05]
 [9.9994695e-01]]
EfficientNet Predictions (first 5):
[[0.00144779]
 [0.99391127]
 [0.9653318 ]
 [0.04704069]
 [0.6775769 ]]
DenseNet121 Probabilities (first 5):
[[0.5   ]
 [0.731 ]
 [0.731 ]
 [0.5005]
 [0.731 ]]
ResNet150 Probabilities (first 5):
[[0.5000143 ]
 [0.73105323]
 [0.73100054]
 [0.5000203 ]
 [0.73104817]]
EfficientNet Probabilities (first 5):
[[0.5003619 ]
 [0.72985977]
 [0.7241881 ]
 [0.51175797]
 [0.66319764]]
Ensemble Probabilities (first 5):
[[0.5001956 ]
 [0.73024476]
 [0.7268312 ]
 [0.5071565 ]
 [0.69024646]]
Ensemble Binary Predictions (first 5):
[[1]
 [1]
 [1]
 [1]
 [1]]
Ensemble Model Accuracy: 49.97%


DenseNet121 & Resnet150 Stacked  ensemble

In [32]:
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

# Assuming models are already loaded (DenseNet121, ResNet150, EfficientNetB0)


# ==================== Predictions from Models ==================== #
# Get predictions for each model on the test dataset
densenet121_predictions = densenet121_model.predict(test_dataset, verbose=1)
resnet150_predictions = resnet150_model.predict(test_dataset, verbose=1)

# Print first few predictions from each model to check for diversity
print("DenseNet121 Predictions (first 5):")
print(densenet121_predictions[:5])

print("ResNet150 Predictions (first 5):")
print(resnet150_predictions[:5])



# ==================== Convert Logits to Probabilities (if necessary) ==================== #
# If models output logits, apply sigmoid to get probabilities
densenet121_probabilities = tf.sigmoid(densenet121_predictions).numpy()
resnet150_probabilities = tf.sigmoid(resnet150_predictions).numpy()


# Print first few probabilities to see the model output
print("DenseNet121 Probabilities (first 5):")
print(densenet121_probabilities[:5])

print("ResNet150 Probabilities (first 5):")
print(resnet150_probabilities[:5])


# ==================== Prepare Stacked Predictions ==================== #
# Stack the model probabilities together (use the probabilities as features)
stacked_predictions = np.column_stack((
    densenet121_probabilities.flatten(),
    resnet150_probabilities.flatten(),

))

# Print first few stacked predictions to see the combined output
print("Stacked Predictions (first 5):")
print(stacked_predictions[:5])

# ==================== Train the Meta-Model (Logistic Regression) ==================== #
# Assuming the true labels are available
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Train a meta-model (Logistic Regression) on the stacked predictions
meta_model = LogisticRegression()
meta_model.fit(stacked_predictions, true_labels)

# ==================== Make Predictions with the Meta-Model ==================== #
# Use the meta-model to predict the final output
ensemble_predictions = meta_model.predict(stacked_predictions)

# Print first few ensemble predictions
print("Ensemble Predictions (first 5):")
print(ensemble_predictions[:5])

# ==================== Evaluate the Ensemble Model ==================== #
# Evaluate the accuracy of the ensemble model
ensemble_accuracy = accuracy_score(true_labels, ensemble_predictions)
print(f'Ensemble Model Accuracy with Stacking: {ensemble_accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step
DenseNet121 Predictions (first 5):
[[2.216e-04]
 [1.000e+00]
 [9.990e-01]
 [2.157e-03]
 [1.000e+00]]
ResNet150 Predictions (first 5):
[[5.7230256e-05]
 [9.9997306e-01]
 [9.9970502e-01]
 [8.1271879e-05]
 [9.9994695e-01]]
DenseNet121 Probabilities (first 5):
[[0.5   ]
 [0.731 ]
 [0.731 ]
 [0.5005]
 [0.731 ]]
ResNet150 Probabilities (first 5):
[[0.5000143 ]
 [0.73105323]
 [0.73100054]
 [0.5000203 ]
 [0.73104817]]
Stacked Predictions (first 5):
[[0.5        0.5000143 ]
 [0.73095703 0.73105323]
 [0.73095703 0.73100054]
 [0.5004883  0.5000203 ]
 [0.73095703 0.73104817]]
Ensemble Predictions (first 5):
[0 1 1 0 1]
Ensemble Model Accuracy with Stacking: 100.00%


**DenseNet121 Resnet150 Resnet50**

In [33]:
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming models are already loaded (DenseNet121, ResNet150, EfficientNetB0)


# ==================== Predictions from Models ==================== #
# Get predictions for each model on the test dataset
densenet121_predictions = densenet121_model.predict(test_dataset, verbose=1)
resnet150_predictions = resnet150_model.predict(test_dataset, verbose=1)
resnet50_predictions = resnet50_model.predict(test_dataset, verbose=1)

# Print first few predictions from each model to check for diversity
print("DenseNet121 Predictions (first 5):")
print(densenet121_predictions[:5])

print("ResNet150 Predictions (first 5):")
print(resnet150_predictions[:5])

print("EfficientNet Predictions (first 5):")
print(resnet50_predictions[:5])

# ==================== Convert Logits to Probabilities (if necessary) ==================== #
# If models output logits, apply sigmoid to get probabilities
densenet121_probabilities = tf.sigmoid(densenet121_predictions).numpy()
resnet150_probabilities = tf.sigmoid(resnet150_predictions).numpy()
resnet50_probabilities = tf.sigmoid(resnet50_predictions).numpy()

# Print first few probabilities to see the model output
print("DenseNet121 Probabilities (first 5):")
print(densenet121_probabilities[:5])

print("ResNet150 Probabilities (first 5):")
print(resnet150_probabilities[:5])

print("EfficientNet Probabilities (first 5):")
print(efficientnet_probabilities[:5])

# ==================== Prepare Stacked Predictions ==================== #
# Stack the model probabilities together (use the probabilities as features)
stacked_predictions = np.column_stack((
    densenet121_probabilities.flatten(),
    resnet150_probabilities.flatten(),
    resnet50_probabilities.flatten()
))

# Print first few stacked predictions to see the combined output
print("Stacked Predictions (first 5):")
print(stacked_predictions[:5])

# ==================== Train the Meta-Model (Logistic Regression) ==================== #
# Assuming the true labels are available
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Train a meta-model (Logistic Regression) on the stacked predictions
meta_model = LogisticRegression()
meta_model.fit(stacked_predictions, true_labels)

# ==================== Make Predictions with the Meta-Model ==================== #
# Use the meta-model to predict the final output
ensemble_predictions = meta_model.predict(stacked_predictions)

# Print first few ensemble predictions
print("Ensemble Predictions (first 5):")
print(ensemble_predictions[:5])

# ==================== Evaluate the Ensemble Model ==================== #
# Evaluate the accuracy of the ensemble model
ensemble_accuracy = accuracy_score(true_labels, ensemble_predictions)
print(f'Ensemble Model Accuracy with Stacking: {ensemble_accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step
DenseNet121 Predictions (first 5):
[[2.216e-04]
 [1.000e+00]
 [9.990e-01]
 [2.157e-03]
 [1.000e+00]]
ResNet150 Predictions (first 5):
[[5.7230256e-05]
 [9.9997306e-01]
 [9.9970502e-01]
 [8.1271879e-05]
 [9.9994695e-01]]
EfficientNet Predictions (first 5):
[[0.001518]
 [1.      ]
 [0.998   ]
 [0.03119 ]
 [1.      ]]
DenseNet121 Probabilities (first 5):
[[0.5   ]
 [0.731 ]
 [0.731 ]
 [0.5005]
 [0.731 ]]
ResNet150 Probabilities (first 5):
[[0.5000143 ]
 [0.73105323]
 [0.73100054]
 [0.5000203 ]
 [0.73104817]]
EfficientNet Probabilities (first 5):
[[0.5003619 ]
 [0.72985977]
 [0.7241881 ]
 [0.51175797]
 [0.66319764]]
Stacked Predictions (first 5):
[[0.5        0.5000143  0.5004883 ]
 [0.73095703 0.73105323 0.73095703]
 [0.73095703 0.73100054 0.73046875]
 [0.5004883  0.5000203  0.5078125 ]
 [0.73095703 0.73104817 0.73095703]]
Ensemble Predictions (first 5):
[0 1 1 0 1]
Ense

In [34]:
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming models are already loaded (DenseNet121, ResNet150, EfficientNetB0)


# ==================== Predictions from Models ==================== #
# Get predictions for each model on the test dataset
densenet121_predictions = densenet121_model.predict(test_dataset, verbose=1)
resnet150_predictions = resnet150_model.predict(test_dataset, verbose=1)
efficientnet_predictions = efficientnetb0_model.predict(test_dataset, verbose=1)

# Print first few predictions from each model to check for diversity
print("DenseNet121 Predictions (first 5):")
print(densenet121_predictions[:5])

print("ResNet150 Predictions (first 5):")
print(resnet150_predictions[:5])

print("EfficientNet Predictions (first 5):")
print(efficientnet_predictions[:5])

# ==================== Convert Logits to Probabilities (if necessary) ==================== #
# If models output logits, apply sigmoid to get probabilities
densenet121_probabilities = tf.sigmoid(densenet121_predictions).numpy()
resnet150_probabilities = tf.sigmoid(resnet150_predictions).numpy()
efficientnet_probabilities = tf.sigmoid(efficientnet_predictions).numpy()

# Print first few probabilities to see the model output
print("DenseNet121 Probabilities (first 5):")
print(densenet121_probabilities[:5])

print("ResNet150 Probabilities (first 5):")
print(resnet150_probabilities[:5])

print("EfficientNet Probabilities (first 5):")
print(efficientnet_probabilities[:5])

# ==================== Prepare Stacked Predictions ==================== #
# Stack the model probabilities together (use the probabilities as features)
stacked_predictions = np.column_stack((
    densenet121_probabilities.flatten(),
    resnet150_probabilities.flatten(),
    efficientnet_probabilities.flatten()
))

# Print first few stacked predictions to see the combined output
print("Stacked Predictions (first 5):")
print(stacked_predictions[:5])

# ==================== Train the Meta-Model (Logistic Regression) ==================== #
# Assuming the true labels are available
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Train a meta-model (Logistic Regression) on the stacked predictions
meta_model = LogisticRegression()
meta_model.fit(stacked_predictions, true_labels)

# ==================== Make Predictions with the Meta-Model ==================== #
# Use the meta-model to predict the final output
ensemble_predictions = meta_model.predict(stacked_predictions)

# Print first few ensemble predictions
print("Ensemble Predictions (first 5):")
print(ensemble_predictions[:5])

# ==================== Evaluate the Ensemble Model ==================== #
# Evaluate the accuracy of the ensemble model
ensemble_accuracy = accuracy_score(true_labels, ensemble_predictions)
print(f'Ensemble Model Accuracy with Stacking: {ensemble_accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step
DenseNet121 Predictions (first 5):
[[2.216e-04]
 [1.000e+00]
 [9.990e-01]
 [2.157e-03]
 [1.000e+00]]
ResNet150 Predictions (first 5):
[[5.7230256e-05]
 [9.9997306e-01]
 [9.9970502e-01]
 [8.1271879e-05]
 [9.9994695e-01]]
EfficientNet Predictions (first 5):
[[0.00144779]
 [0.99391127]
 [0.9653318 ]
 [0.04704069]
 [0.6775769 ]]
DenseNet121 Probabilities (first 5):
[[0.5   ]
 [0.731 ]
 [0.731 ]
 [0.5005]
 [0.731 ]]
ResNet150 Probabilities (first 5):
[[0.5000143 ]
 [0.73105323]
 [0.73100054]
 [0.5000203 ]
 [0.73104817]]
EfficientNet Probabilities (first 5):
[[0.5003619 ]
 [0.72985977]
 [0.7241881 ]
 [0.51175797]
 [0.66319764]]
Stacked Predictions (first 5):
[[0.5        0.5000143  0.5003619 ]
 [0.73095703 0.73105323 0.72985977]
 [0.73095703 0.73100054 0.7241881 ]
 [0.5004883  0.5000203  0.51175797]
 [0.73095703 0.73104817 0.66319764]]
Ensemble Predictions (first 5):
[0 1 1

Best 5 models

In [35]:
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming models are already loaded (DenseNet121, ResNet150, EfficientNetB0)


# ==================== Predictions from Models ==================== #
# Get predictions for each model on the test dataset
densenet121_predictions = densenet121_model.predict(test_dataset, verbose=1)
resnet150_predictions = resnet150_model.predict(test_dataset, verbose=1)
resnet50_predictions = resnet50_model.predict(test_dataset, verbose=1)
efficientnetb0_predictions = efficientnetb0_model.predict(test_dataset, verbose=1)
efficientnetb1_predictions = efficientnetb1_model.predict(test_dataset, verbose=1)


# Print first few predictions from each model to check for diversity
print("DenseNet121 Predictions (first 5):")
print(densenet121_predictions[:5])

print("ResNet150 Predictions (first 5):")
print(resnet150_predictions[:5])

print("EfficientNet Predictions (first 5):")
print(efficientnet_predictions[:5])

# ==================== Convert Logits to Probabilities (if necessary) ==================== #
# If models output logits, apply sigmoid to get probabilities
densenet121_probabilities = tf.sigmoid(densenet121_predictions).numpy()
resnet150_probabilities = tf.sigmoid(resnet150_predictions).numpy()
resnet50_probabilities = tf.sigmoid(resnet50_predictions).numpy()
efficientnetb0_probabilities = tf.sigmoid(efficientnetb0_predictions).numpy()
efficientnetb1_probabilities = tf.sigmoid(efficientnetb1_predictions).numpy()
# Print first few probabilities to see the model output
print("DenseNet121 Probabilities (first 5):")
print(densenet121_probabilities[:5])

print("ResNet150 Probabilities (first 5):")
print(resnet150_probabilities[:5])

print("EfficientNet Probabilities (first 5):")
print(efficientnet_probabilities[:5])

# ==================== Prepare Stacked Predictions ==================== #
# Stack the model probabilities together (use the probabilities as features)
stacked_predictions = np.column_stack((
    densenet121_probabilities.flatten(),
    resnet150_probabilities.flatten(),
    resnet50_probabilities.flatten(),
    efficientnetb0_probabilities.flatten(),
    efficientnetb1_probabilities.flatten()
))

# Print first few stacked predictions to see the combined output
print("Stacked Predictions (first 5):")
print(stacked_predictions[:5])

# ==================== Train the Meta-Model (Logistic Regression) ==================== #
# Assuming the true labels are available
true_labels = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)

# Train a meta-model (Logistic Regression) on the stacked predictions
meta_model = LogisticRegression()
meta_model.fit(stacked_predictions, true_labels)

# ==================== Make Predictions with the Meta-Model ==================== #
# Use the meta-model to predict the final output
ensemble_predictions = meta_model.predict(stacked_predictions)

# Print first few ensemble predictions
print("Ensemble Predictions (first 5):")
print(ensemble_predictions[:5])

# ==================== Evaluate the Ensemble Model ==================== #
# Evaluate the accuracy of the ensemble model
ensemble_accuracy = accuracy_score(true_labels, ensemble_predictions)
print(f'Ensemble Model Accuracy with Stacking: {ensemble_accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 27s 231ms/step
DenseNet121 Predictions (first 5):
[[2.216e-04]
 [1.000e+00]
 [9.990e-01]
 [2.157e-03]
 [1.000e+00]]
ResNet150 Predictions (first 5):
[[5.7230256e-05]
 [9.9997306e-01]
 [9.9970502e-01]
 [8.1271879e-05]
 [9.9994695e-01]]
EfficientNet Predictions (first 5):
[[0.00144779]
 [0.99391127]
 [0.9653318 ]
 [0.04704069]
 [0.6775769 ]]
DenseNet121 Probabilities (first 5):
[[0.5   ]
 [0.731 ]
 [0.731 ]
 [0.5005]
 [0.731 ]]
ResNet150 Probabilities (first 5):
[[0.5000143 ]
 [0.73105323]
 [0.73100054]
 [0.5000203 ]
 [0.73104817]]
EfficientNet Probabilities (first 5):
[[0.5003619 ]
 [0.72985977]
 [0.7241881 ]
 [0.51175797]
 [0.66319764]]
Stacked Predictions (first 5):
[[0.5        0.5000143  0.5004883  0.5003619  0.5151367 ]
 [0.73095703 0.73105323 0.73095703 0.72985977 0.7260742 ]
 [0.73095703 0.731000

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Create a stacked predictions array
stacked_predictions = np.column_stack((densenet_probabilities, resnet150_probabilities, efficientnet_probabilities))

# Create and train a meta-model (e.g., logistic regression)
meta_model = LogisticRegression()
meta_model.fit(stacked_predictions, true_labels)

# Make predictions with the meta-model
ensemble_predictions = meta_model.predict(stacked_predictions)

# Evaluate the stacked model
accuracy = accuracy_score(true_labels, ensemble_predictions)
print(f'Ensemble Model Accuracy with Stacking: {accuracy * 100:.2f}%')


Ensemble Model Accuracy with Stacking: 100.00%


In [ ]:
# Print the model architecture to find the correct layer name
print(densenet121_model.summary())
# Print the model architecture to find the correct layer name
print(resnet150_model.summary())
# Print the model architecture to find the correct layer name
print(resnet50_model.summary())
# Print the model architecture to find the correct layer name
print(efficientnetb0_model.summary())
# Print the model architecture to find the correct layer name
print(efficientnetb1_model.summary())





Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ densenet121 (Functional)             │ (None, 8, 8, 1024)          │       7,037,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,562,823 (28.85 MB)

 Trainable params: 7,134,081 (27.21 MB)

 Non-trainable params: 428,736 (1.64 MB)

...

 Trainable params: 21,560,641 (82.25 MB)

 Non-trainable params: 17,152 (67.00 KB)

 Optimizer params: 6 (36.00 B)

None


Model: "functional"

In [ ]:
# List all layers in the DenseNet121 model to find the last convolutional layer
for layer in densenet121_model.get_layer('densenet121').layers:
    print(layer.name)


In [ ]:
# List all layers in the 'densenet121' block to find the last convolutional layer
for layer in resnet50_model.get_layer('resnet50').layers:
    print(layer.name)


In [ ]:
'''
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

# Define mean and std for the dataset you trained on (assuming ImageNet stats)
mean = np.array([0.485, 0.456, 0.406])  # Mean for ImageNet
std = np.array([0.229, 0.224, 0.225])   # Std for ImageNet

# Function to apply mean and std normalization (standardization)
def preprocess_single_image(img, target_size=(256, 256)):
    # Resize and convert the image to an array
    img = image.load_img(img, target_size=target_size)
    img_array = image.img_to_array(img)
    
    # Normalize the image using the mean and std of the dataset
    img_array = img_array / 255.0  # Rescale to [0, 1]
    img_array = (img_array - mean) / std  # Apply mean and std normalization
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to generate Grad-CAM
def generate_gradcam(model, img_array, last_conv_layer_name, class_index):
    last_conv_layer = model.get_layer(last_conv_layer_name)

    grad_model = Model(inputs=model.input, outputs=[last_conv_layer.output, model.output])

    with tf.GradientTape() as tape:
        last_conv_layer_output, predictions = grad_model(img_array)
        class_output = predictions[:, class_index]  # Get the class index for Grad-CAM

    gradients = tape.gradient(class_output, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(gradients, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    for i in range(last_conv_layer_output.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(last_conv_layer_output, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    return heatmap

# Function to overlay the heatmap on the image
def overlay_heatmap(heatmap, img_path, alpha=0.6):
    img = image.load_img(img_path, target_size=(256, 256))  # Load the image
    img = image.img_to_array(img)
    
    heatmap = np.uint8(255 * heatmap)  # Convert heatmap to 255 scale
    heatmap = np.expand_dims(heatmap, axis=-1)
    heatmap = np.repeat(heatmap, 3, axis=-1)  # Convert heatmap to 3 channels

    # Apply the heatmap to the image
    superimposed_img = img * (1 - alpha) + heatmap * alpha
    return superimposed_img

# ==================== Grad-CAM for a Single Image ==================== #

# Path to the image you want to visualize
img_path = '/kaggle/input/lung-and-colon-cancer-histopathological-images/lung_colon_image_set/lung_image_sets/lung_scc/lungscc1001.jpeg'  # Replace with your image path

# Preprocess the single image
img_array = preprocess_single_image(img_path)

# Get the model predictions (ensemble or single model)
densenet121_predictions = densenet121_model.predict(img_array)
resnet150_predictions = resnet150_model.predict(img_array)
resnet50_predictions = resnet50_model.predict(img_array)

# Get the class index for the top predicted class (use ensemble prediction or individual model)
class_index = np.argmax(densenet121_predictions)  # Or you can use ensemble model's prediction

# ==================== Layer Names ==================== #
# Access the correct layer names for each model
densenet121_last_conv_layer = "conv5_block12_concat"  # Found after checking DenseNet121 layer list
resnet150_last_conv_layer = "conv5_block3_2_conv"   # Layer for ResNet150 (last conv layer)
resnet50_last_conv_layer = "conv5_block3_3_conv"    # Layer for ResNet50 (last conv layer)

# ==================== Generate Grad-CAM heatmap ==================== #
# Generate Grad-CAM heatmap for each model
densenet121_heatmap = generate_gradcam(densenet121_model, img_array, densenet121_last_conv_layer, class_index)  # Update with correct layer name
resnet150_heatmap = generate_gradcam(resnet150_model, img_array, resnet150_last_conv_layer, class_index)   # Update with correct layer name
resnet50_heatmap = generate_gradcam(resnet50_model, img_array, resnet50_last_conv_layer, class_index)    # Update with correct layer name

# ==================== Overlay Heatmap on Image ==================== #
superimposed_densenet = overlay_heatmap(densenet121_heatmap, img_path)
superimposed_resnet150 = overlay_heatmap(resnet150_heatmap, img_path)
superimposed_resnet50 = overlay_heatmap(resnet50_heatmap, img_path)

# ==================== Display the Results ==================== #
plt.figure(figsize=(10, 10))

# DenseNet121
plt.subplot(1, 3, 1)
plt.imshow(superimposed_densenet / 255.)
plt.title("DenseNet121 Grad-CAM")
plt.axis('off')

# ResNet150
plt.subplot(1, 3, 2)
plt.imshow(superimposed_resnet150 / 255.)
plt.title("ResNet150 Grad-CAM")
plt.axis('off')

# ResNet50
plt.subplot(1, 3, 3)
plt.imshow(superimposed_resnet50 / 255.)
plt.title("ResNet50 Grad-CAM")
plt.axis('off')

plt.show()
'''

In [ ]:
'''
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Load pre-trained DenseNet121 model (or your chosen model)
densenet121_model = tf.keras.applications.DenseNet121(weights='imagenet')

# Function to get the last convolutional layer name
def get_last_conv_layer_name(model):
    conv_layers = []
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Conv2D):  # Check if layer is Conv2D
            conv_layers.append(layer.name)  # Add to list if Conv2D
    return conv_layers[-1] if conv_layers else None

# Get the last convolutional layer name for DenseNet121
last_conv_layer_name = get_last_conv_layer_name(densenet121_model)
print(f"Last Convolutional Layer: {last_conv_layer_name}")

# Function to preprocess the image (resize it to 224x224)
def preprocess_single_image(img_path, target_size=(224, 224)):  # Resize to (224, 224)
    img = image.load_img(img_path, target_size=target_size)  # Resize the image
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Rescale to [0, 1]
    return img_array

# Example image path (replace with the actual path of your image)
img_path = '/kaggle/input/lung-and-colon-cancer-histopathological-images/lung_colon_image_set/lung_image_sets/lung_aca/lungaca1008.jpeg'  # Replace with your image path

# Preprocess the image (resize to 224x224)
img_array = preprocess_single_image(img_path)

# Function to generate Grad-CAM
def generate_gradcam(model, img_array, last_conv_layer_name, class_index):
    last_conv_layer = model.get_layer(last_conv_layer_name)
    grad_model = tf.keras.models.Model(inputs=model.input, outputs=[last_conv_layer.output, model.output])

    with tf.GradientTape() as tape:
        last_conv_layer_output, predictions = grad_model(img_array)
        class_output = predictions[:, class_index]  # Get the class index for Grad-CAM

    gradients = tape.gradient(class_output, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(gradients, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]  # Get the first image in the batch

    # Corrected gradient multiplication
    pooled_grads = tf.reshape(pooled_grads, (1, 1, 1, -1))  # Reshape pooled grads to match feature map channels
    grad_activated_output = last_conv_layer_output * pooled_grads  # Multiply gradients with the feature map

    # Compute the heatmap by averaging across channels
    heatmap = tf.reduce_mean(grad_activated_output, axis=-1)  # Average across channels
    heatmap = tf.maximum(heatmap, 0)  # Apply ReLU
    heatmap = heatmap / tf.reduce_max(heatmap)  # Normalize the heatmap

    return heatmap

# Get the predictions for the image
predictions = densenet121_model.predict(img_array)

# Get the class index for the top predicted class
class_index = np.argmax(predictions)

# Generate Grad-CAM heatmap for DenseNet121
densenet121_heatmap = generate_gradcam(densenet121_model, img_array, last_conv_layer_name, class_index)

# Function to overlay the heatmap on the image
def overlay_heatmap(heatmap, img_path, alpha=0.6):
    img = image.load_img(img_path, target_size=(224, 224))  # Resize the image to 224x224
    img = image.img_to_array(img)

    # Resize heatmap to match the original image size (224x224)
    heatmap = tf.image.resize(heatmap, (224, 224))  # Resize the heatmap to 224x224
    print(f"Heatmap shape after resizing: {heatmap.shape}")  # Debugging resized heatmap shape

    # Normalize heatmap and convert it to a 3-channel image (RGB)
    heatmap = np.uint8(255 * heatmap)  # Convert heatmap to 255 scale
    heatmap = np.expand_dims(heatmap, axis=-1)  # Add channel dimension (224, 224, 1)
    heatmap = np.repeat(heatmap, 3, axis=-1)  # Repeat the heatmap for 3 channels (RGB)

    # Apply the heatmap to the image
    superimposed_img = img * (1 - alpha) + heatmap * alpha
    return superimposed_img

# Overlay heatmap on the image
superimposed_img = overlay_heatmap(densenet121_heatmap, img_path)

# Display the image with Grad-CAM heatmap
plt.imshow(superimposed_img / 255.)
plt.title("DenseNet121 Grad-CAM")
plt.axis('off')
plt.show()
'''